# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902041


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:08<01:57,  4.20s/it]

Rendering models:  10%|█         | 3/30 [00:09<01:30,  3.36s/it]

Rendering models:  13%|█▎        | 4/30 [00:10<01:04,  2.47s/it]

Rendering models:  17%|█▋        | 5/30 [00:11<00:50,  2.01s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:27<00:03,  1.24it/s]

Rendering models:  90%|█████████ | 27/30 [00:29<00:03,  1.05s/it]

Rendering models:  93%|█████████▎| 28/30 [00:30<00:02,  1.10s/it]

Rendering models: 100%|██████████| 30/30 [00:31<00:00,  1.09it/s]

esazure                               0.002422
BeNjEsSePh                            0.000819
kayleebug2017                         0.001238
equidad1                              0.003835
not-logged-in-134238199d806cd7ef54    0.014615
Thedunkmasta                          0.002425
not-logged-in-7526601586eaa938b3ee    0.001581
ElisabethB                            0.039573
Ella7777                              0.002340
Linda_J._Berkel                       0.000669
not-logged-in-6cd9487f1f13f15d1751    0.132960
redboy21                              0.462834
lattecuc                              0.000923
sn346808                              0.019296
Rmarks1701                            0.000950
Jakub21                               0.001710
not-logged-in-ffc5c906399855f5ef0f    0.000783
not-logged-in-623cb7cd00f5dbcaf5ac    0.000963
puic                                  0.003609
not-logged-in-e84bc26c80a36790c29a    0.011421
nthibert1                             0.007660
H_Robinson   

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())